# A* Search Implementation for the Romanian Map Problem

In this exercise you are ask to implement the A* search algorithm in Python to find the shortest path from Arad to Bucharest on the Romanian map.

## Step 1: Representing the Graph and Heuristics

Before we write the algorithm, we need to store the map data. 
We can use a dictionary (as an adjacency list) for the map connections and another dictionary for the heuristic values (straight-line distances to Bucharest).

In [ ]:
#
# STEP 1: Define the graph and heuristic data
#

# The romania_map is represented as an adjacency list.
# The keys are city names, and the values are dictionaries of neighboring cities
# and the distance to them. This is a bi-directional graph.
romania_map = {
    'Arad': {'Zerind': 75, 'Sibiu': 140, 'Timisoara': 118},
    'Zerind': {'Arad': 75, 'Oradea': 71},
    'Oradea': {'Zerind': 71, 'Sibiu': 151},
    'Sibiu': {'Arad': 140, 'Oradea': 151, 'Fagaras': 99, 'Rimnicu Vilcea': 80},
    'Timisoara': {'Arad': 118, 'Lugoj': 111},
    'Lugoj': {'Timisoara': 111, 'Mehadia': 70},
    'Mehadia': {'Lugoj': 70, 'Drobeta': 75},
    'Drobeta': {'Mehadia': 75, 'Craiova': 120},
    'Craiova': {'Drobeta': 120, 'Rimnicu Vilcea': 146, 'Pitesti': 138},
    'Rimnicu Vilcea': {'Sibiu': 80, 'Craiova': 146, 'Pitesti': 97},
    'Fagaras': {'Sibiu': 99, 'Bucharest': 211},
    'Pitesti': {'Rimnicu Vilcea': 97, 'Craiova': 138, 'Bucharest': 101},
    'Bucharest': {'Fagaras': 211, 'Pitesti': 101, 'Giurgiu': 90, 'Urziceni': 85},
    'Giurgiu': {'Bucharest': 90},
    'Urziceni': {'Bucharest': 85, 'Hirsova': 98, 'Vaslui': 142},
    'Hirsova': {'Urziceni': 98, 'Eforie': 86},
    'Eforie': {'Hirsova': 86},
    'Vaslui': {'Urziceni': 142, 'Iasi': 92},
    'Iasi': {'Vaslui': 92, 'Neamt': 87},
    'Neamt': {'Iasi': 87}
}

# Heuristic values: Straight-line distance to Bucharest
heuristic_distances = {
    'Arad': 366,
    'Bucharest': 0,
    'Craiova': 160,
    'Drobeta': 242,
    'Eforie': 161,
    'Fagaras': 176,
    'Giurgiu': 77,
    'Hirsova': 151,
    'Iasi': 226,
    'Lugoj': 244,
    'Mehadia': 241,
    'Neamt': 234,
    'Oradea': 380,
    'Pitesti': 100,
    'Rimnicu Vilcea': 193,
    'Sibiu': 253,
    'Timisoara': 329,
    'Urziceni': 80,
    'Vaslui': 199,
    'Zerind': 374
}

## Step 2: Implementing the A* Algorithm 

Now, we'll implement the A* search function. 
We can use Python's heapq library, which provides an efficient implementation of a min-priority queue.

In [ ]:
import heapq
import math

#
# STEP 2: Implement the A* Search Algorithm
#

def a_star_search(graph, start, goal, heuristics):
    """
    Finds the shortest path from start to goal using A* search.
    graph: dict[node] -> dict[neighbor] = edge_cost
    heuristics: dict[node] = h(node)
    Returns a list of nodes for the path, or None if no path.
    """
    # The set of nodes already evaluated.
    explored_set = set()

    # The priority queue of nodes to be evaluated. Stores tuples of (f_score, tie, node).
    # The tie-breaker avoids comparing nodes when f_scores are equal.
    frontier = []
    counter = 0

    # A dictionary to reconstruct the path. came_from[n] is the node immediately
    # preceding n on the cheapest path from start to n currently known.
    came_from = {}

    # g_score[n] is the cost of the cheapest path from start to n known so far.
    g_score = {start: 0}

    h_start = heuristics.get(start, 0)
    heapq.heappush(frontier, (g_score[start] + h_start, counter, start))

    while frontier:
        # Get the node in the frontier with the lowest f_score.
        # The f_score is not needed after this, so we use '_'
        _, _, current_node = heapq.heappop(frontier)

        # If we have reached the goal, reconstruct and return the path.
        if current_node == goal:
            return reconstruct_path(came_from, current_node)

        if current_node in explored_set:
            continue
        explored_set.add(current_node)

        # Explore neighbors
        for neighbor, step_cost in graph.get(current_node, {}).items():
            if neighbor in explored_set:
                continue

            # Calculate the tentative g_score for the path through the current_node
            tentative_g = g_score[current_node] + step_cost

            # If this path to the neighbor is better than any previously known...
            if tentative_g < g_score.get(neighbor, math.inf):
                # ...record it as the new best path.
                came_from[neighbor] = current_node
                g_score[neighbor] = tentative_g

                # Add/update the neighbor in the frontier for evaluation.
                h = heuristics.get(neighbor, 0)
                counter += 1
                heapq.heappush(frontier, (tentative_g + h, counter, neighbor))

    # If the frontier is empty and we haven't reached the goal, no path exists.
    return None

def reconstruct_path(came_from, current_node):
    """
    Reconstructs the path from the came_from dictionary.
    """
    path = [current_node]
    while current_node in came_from:
        current_node = came_from[current_node]
        path.append(current_node)
    return path[::-1]  # Reverse the path to get it from start to goal

def calculate_path_cost(graph, path):
    """
    Calculates the total cost of a given path.
    """
    if not path or len(path) < 2:
        return 0
    cost = 0
    for i in range(len(path) - 1):
        cost += graph[path[i]][path[i+1]]
    return cost

In [ ]:
# Define start and goal cities
start_city = 'Arad'
goal_city = 'Bucharest'

# Run A* search
path = a_star_search(romania_map, start_city, goal_city, heuristic_distances)

# Print the results
if path:
    print("A* Path from", start_city, "to", goal_city, ":")
    print(" → ".join(path))
    total_cost = calculate_path_cost(romania_map, path)
    print(f"\nTotal path cost: {total_cost} km")
else:
    print(f"No path found from {start_city} to {goal_city}.")
